The purpose of this notebook is to identify the frequency of conflicts between metrics that purport to measure the degree to which a plan favors one party or another.

In [1]:
from typing import List, Dict, Any, Set

import os
import pandas as pd
from collections import defaultdict

from rdametrics import states, chambers, ensembles

Load the scores dataframe

In [2]:
scores_path: str = "~/local/beta-ensembles/prepackaged/scores/scores.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))

Helper code

In [13]:
# from typing import Union
import numpy as np
import pandas as pd

ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

partisan_bias = {
    "partisan_advantage": ["disproportionality", "efficiency_gap"],
    "partisan_symmetry": ["seats_bias", "votes_bias", "geometric_seats_bias"],
    "packing_cracking": ["mean_median_average_district", "lopsided_outcomes", "declination"],
}

categories: List[str] = list(partisan_bias.keys())

# def same_sign(a: float | int | None, b: float | int | None) -> bool:
#     if a == 0.0 or b == 0.0:
#         return True
    
#     if pd.isna(a) or pd.isna(b):
#         return True

#     return a * b > 0

def same_sign(a, b):
    if a == 0.0 or b == 0.0:
        return True
    
    if pd.isna(a) or pd.isna(b):
        return True
    
    assert isinstance(a, (int, float)) and isinstance(b, (int, float)), f"Unexpected types: {type(a)}, {type(b)}"

    return a * b > 0

# def same_signs(list1, list2):
#     # Combine both lists and filter out zero values and NaN values
#     all_values = list1 + list2
#     non_zero_defined_values = [x for x in all_values if x != 0 and not pd.isna(x)]
    
#     # If no valid values or only one valid value, return True
#     if len(non_zero_defined_values) <= 1:
#         return True
    
#     # Check if all valid values have the same sign
#     all_positive = all(x > 0 for x in non_zero_defined_values)
#     all_negative = all(x < 0 for x in non_zero_defined_values)
    
#     return all_positive or all_negative



Setup counters for various kinds of conflicts.

In [18]:
from itertools import combinations
import copy
from collections import defaultdict

by_state: Dict[str, Any] = dict()

for xx in states:
    by_state[xx] = {
        "Vf": None,
        "total": 0
    }

conflicts = dict()
_ledger = {
    "counts": defaultdict(int),
    "combos": set(),
    "example": None,
    "value1": None,
    "value2": None,
    "delta": None,
}

for c in categories:
    conflicts[c] = dict()
    pairs = combinations(partisan_bias[c], 2)
    for p in pairs:
        conflicts[c][p] = copy.deepcopy(_ledger)

conflicts["cross_category"] = dict()
pairs = combinations(categories, 2)
for p in pairs:
    conflicts["cross_category"][p] = copy.deepcopy(_ledger)

# conflicts

Count instances where the scores conflict. Keep track by state / chamber / ensemble combination.

In [ ]:
for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    by_state[xx]["total"] += 1
    if by_state[xx]["Vf"] is None:
        by_state[xx]["Vf"] = row["estimated_vote_pct"]

    consistent = {
        "partisan_advantage": True,
        "packing_cracking": True,
        "partisan_symmetry": True
    }

    # Check consistency within each category

    for c in categories:
        for _pair, _ledger in conflicts[c].items():
            m1, m2 = _pair
            v1 = row[m1]
            v2 = row[m2]
            delta = abs(v1 - v2)
            if not same_sign(v1, v2):
                consistent[c] = False
                _ledger["counts"][xx] += 1
                _ledger["combos"].add(combo)
                if _ledger["example"] is None or delta > _ledger["delta"]:
                    _ledger["example"] = combo
                    _ledger["value1"] = v1
                    _ledger["value2"] = v2
                    _ledger["delta"] = delta

    # Compare consistency across categories
    for _pair, _ledger in conflicts["cross_category"].items():
        cat1, cat2 = _pair
        if consistent[cat1] and consistent[cat2]:
            v1 = row[partisan_bias[cat1][0]]
            v2 = row[partisan_bias[cat2][0]]
            if not same_sign(v1, v2):
                _ledger["counts"][xx] += 1
                _ledger["combos"].add(combo)
                if _ledger["example"] is None:
                    _ledger["example"] = combo
                    # _ledger["value1"] = cat1
                    # _ledger["value2"] = cat2
                    # _ledger["delta"] = None


# (by_state, conflicts)
# conflicts

Aggregate the results into a reportable format.

In [ ]:
def partisan_balance(Vf: float) -> float:
    balance: float = Vf - 0.5
    if Vf < 0.5:
        balance = -balance
    return balance

report = dict()

for c in categories + ["cross_category"]:
    report[c] = dict()
    for _pair, _ledger in conflicts[c].items():
        p1, p2 = _pair

        diffs_by_state = [(xx, partisan_balance(by_state[xx]['Vf']), count / by_state[xx]['total']) for xx, count in _ledger["counts"].items()]
        diffs_by_state.sort(key=lambda x: x[1])

        diffs = [(xx, f"{Vf:.2%}", f"{rate:.2%}") for xx, Vf, rate in diffs_by_state]

        report[c][(p1, p2)] = {
            "diff-rate": diffs,
            "combos": len(_ledger["combos"]),
            "example": _ledger["example"],
            "value1": _ledger["value1"],
            "value2": _ledger["value2"],
            "delta": _ledger["delta"],
        }



report

{'partisan_advantage': {('disproportionality',
   'efficiency_gap'): {'diff-rate': [('NC', '0.57%', '1.04%'),
    ('WI', '0.68%', '0.35%'),
    ('FL', '1.63%', '2.94%'),
    ('MI', '1.88%', '2.06%'),
    ('OH', '3.62%', '0.74%'),
    ('IL', '8.17%', '71.68%'),
    ('NY', '14.78%', '24.33%')], 'combos': 165, 'example': ('NY',
    'upper',
    'A0'), 'value1': -0.1428, 'value2': 0.005, 'delta': 0.14780000000000001}},
 'partisan_symmetry': {('seats_bias', 'votes_bias'): {'diff-rate': [],
   'combos': 0,
   'example': None,
   'value1': None,
   'value2': None,
   'delta': None},
  ('seats_bias',
   'geometric_seats_bias'): {'diff-rate': [('NC', '0.57%', '0.03%'),
    ('WI', '0.68%', '0.01%'),
    ('FL', '1.63%', '3.60%'),
    ('MI', '1.88%', '0.01%'),
    ('OH', '3.62%', '2.73%'),
    ('IL', '8.17%', '0.02%'),
    ('NY', '14.78%', '0.14%')], 'combos': 132, 'example': ('NY',
    'congress',
    'D'), 'value1': 0.093, 'value2': -0.0066, 'delta': 0.0996},
  ('votes_bias',
   'geometric_seats